In [1]:
!pip install unsloth #install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git #also get the latest version of unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 7.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 56.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━

In [2]:
#Models for fine-tuning 
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer #Trainer for supervised fine-tuning
from unsloth import is_bfloat16_supported #checks if the hardware supports bfloat16 precision
#Hugging Face models
from huggingface_hub import login #Lets you login to API
from transformers import TrainingArguments #Defines training hyperparameters
from datasets import load_dataset #Lets you load fine-tuning datasets
#import weights and biases
import wandb
#Import kaggle secrets
from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-27 09:10:44.620963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751015444.844137      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751015444.909165      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


# Creat API kyes and login HuggingFace and Weights and Biases

In [3]:
#Initialize huggingface & wandb tokens
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HG_TOKEN")
wandb_token = user_secrets.get_secret("wab")
#login to huggingface
login(hf_token)
#login to weights-and-biases
wandb.login(key=wandb_token)
run = wandb.init(
    project = 'Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset',
    job_type = "training",
    anonymous= 'allow'    
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdelrhmanessam829 (abdelrhmanessam829-a-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Load DeepSeek and the Tokenizer

In [4]:
#Set params
max_seq_length = 2048
dtype = None
load_in_4bit = True
#Load the DeepSeek R1 model and tokenizer using unsloth 
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.6.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

# Testing DeepSeek-R1 on a medical use-case before fine-tuning

In [5]:
#Define a system prompt
prompt = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before Answering. thing carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

# Running inference on the model

In [6]:
# Create a test medical question for inference
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
            sneezing but no leakage at night undergoes a gynocological exam and Q-tipe test. Base on these findings,
            what would cystometry most likely reveal about her residual volume and detrusor contractions?"""
# Enable optimized inference mode for unsloth models
FastLanguageModel.for_inference(model)
#Format the question using the structured prompt and tokenize it
inputs = tokenizer([prompt.format(question, "")] , return_tensors = "pt").to("cuda")
#Generate a response using the model
outputs = model.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)
#Decode the generate output tokens into human readable text
response = tokenizer.batch_decode(outputs)
#Extract the response part
print(response[0].split("### Response")[1])

:
<think>
Okay, so I need to figure out what a cystometry would show for this 61-year-old woman. Let's start by breaking down the problem.

First, the patient has a history of involuntary urine loss, specifically when she coughs or sneezes. That makes me think of stress urinary incontinence. But she doesn't leak at night, so it's more likely to be a functional issue rather than something like nocturnal enuresis, which is usually from a neurological problem.

She underwent a gynecological exam and a Q-tip test. I remember that the Q-tip test is used to assess urethral function. The Q-tip is inserted into the urethra, and the patient is asked to cough. If the Q-tip remains in the vagina, it suggests that the urethral sphincter is functioning properly, meaning it's closing during a cough. If it pops out, it might indicate a weakness in the sphincter.

Now, considering cystometry. Cystometry is a diagnostic test where a catheter is placed in the bladder, and the patient is asked to cough o

# Fine-tuning

In [7]:
# Updated training prompt style 
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before Answering. thing carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

# Donwload the fine-tuning dataset and format it for fine-tuning

In [8]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT" , "en" ,split= "train[0:500]" , trust_remote_code = True)
dataset

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 500
})

In [9]:
dataset[1]

{'Question': 'A 33-year-old woman is brought to the emergency department 15 minutes after being stabbed in the chest with a screwdriver. Given her vital signs of pulse 110/min, respirations 22/min, and blood pressure 90/65 mm Hg, along with the presence of a 5-cm deep stab wound at the upper border of the 8th rib in the left midaxillary line, which anatomical structure in her chest is most likely to be injured?',
 'Complex_CoT': "Okay, let's figure out what's going on here. A woman comes in with a stab wound from a screwdriver. It's in her chest, upper border of the 8th rib, left side, kind of around the midaxillary line. First thought, that's pretty close to where the lung sits, right?\n\nLet's talk about location first. This spot is along the left side of her body. Above the 8th rib, like that, is where a lot of important stuff lives, like the bottom part of the left lung, possibly the diaphragm too, especially considering how deep the screwdriver went.\n\nThe wound is 5 cm deep. Tha

In [10]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [11]:
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]

    texts = []

    for input , cot , output in zip(inputs , cots , outputs):
        text = train_prompt_style.format(input , cot , output) + EOS_TOKEN
        texts.append(text)

    return {
        "text": texts
    }

In [12]:
dataset_finetune = dataset.map(formatting_prompts_func , batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore Answering. thing carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swo

# Setting up the model using LoRA

In [13]:
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.6.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=dataset_finetune,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

# Model Training

In [15]:
#Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.968800
20,1.426000
30,1.404400
40,1.360300
50,1.390200
60,1.369300


In [16]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.6707678365417472e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.25737
train/learning_rate,0.0
train/loss,1.3693


In [17]:
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
            sneezing but no leakage at night undergoes a gynocological exam and Q-tipe test. Base on these findings,
            what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

#Load the inference model
FastLanguageModel.for_inference(model_lora)
#tokenize the input question with a specific prompt format
inputs = tokenizer([prompt.format(question, "")] , return_tensors = "pt").to("cuda")
#Generate a response using the model
outputs = model_lora.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)
#Decode the generate output tokens into human readable text
response = tokenizer.batch_decode(outputs)
#Extract the response part
print(response[0].split("### Response")[1])

:
<think>
Okay, let's see. She's a 61-year-old woman with a history of involuntary urine loss, especially when she coughs or sneezes. That sounds like a classic case of stress incontinence. I remember that stress incontinence is usually caused by weak muscles in the pelvic floor, which can't hold the bladder when there's pressure.

Now, when they mentioned she doesn't leak at night, it makes me think she doesn't have the urge to urinate at night, which is more typical of urge incontinence. That's interesting because it suggests stress incontinence is the primary issue here.

They also mentioned she had a gynecological exam and a Q-tip test. I think the Q-tip test is used to check the position of the urethra and how well it's supported. If the urethra is poorly supported, it's more likely to cause stress incontinence.

So, what do we expect from a cystometry in this case? Well, in stress incontinence, the bladder doesn't empty completely because the urethral closure isn't strong enough.